In [12]:
print("Ok")

Ok


In [13]:
%pwd

'c:\\Users\\spjha\\OneDrive\\Desktop\\Hackathon'

In [2]:
import os
os.chdir("../")

In [15]:
%pwd

'c:\\Users\\spjha\\OneDrive\\Desktop'

In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [ ]:
extracted_data = load_pdf_file(data='data/')


NameError: name 'load_pdf_file' is not defined

In [ ]:
# extracted_data

In [ ]:
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [ ]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1879


In [ ]:
# text_chunks

In [19]:

from langchain.embeddings import HuggingFaceEmbeddings

In [20]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [21]:
embeddings = download_hugging_face_embeddings()

C:\Users\spjha\AppData\Local\Temp\ipykernel_5732\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\spjha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\spjha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Pyth

In [22]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [ ]:
#query_result

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os

print("Current working directory:", os.getcwd())
print("Does .env exist here?", os.path.isfile(".env"))


Current working directory: c:\Users\spjha\OneDrive\Desktop\Hackathon\research
Does .env exist here? False


In [4]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [7]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "mindease"

if not pc.has_index(index_name):
    pc.create_index_for_model(
        name=index_name,
        dimension=384,
        metric="cosine",
        cloud="aws",
        region="us-east-1",
        embed={
            "model":"llama-text-embed-v2",
            "field_map":{"text": "chunk_text"}
        }
    )

AttributeError: 'PineconeGRPC' object has no attribute 'has_index'

In [ ]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [ ]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [ ]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [ ]:
docsearch

In [ ]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
retrieved_docs = retriever.invoke("What is Anxiety?")

In [ ]:
retrieved_docs

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=os.getenv("GEMINI_API_KEY")
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",  
    temperature=0.4,
    max_output_tokens=500,
    google_api_key=os.getenv("GEMINI_API_KEY") 
)

In [ ]:
print("Gemini Key:", os.getenv("GEMINI_API_KEY"))


In [ ]:
print("Available indexes:", pc.list_indexes().names())

In [ ]:
response = rag_chain.invoke({"input": "why do we need a tranformation in mental health"})
print(response["answer"])

In [ ]:
response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])